In [1]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

from sapaicore_utils import (DOCKER_TOKEN,
                             DOCKER_USER,
                             get_conn_details,
                             GITHUB_TOKEN,
                             GITHUB_USER,
                             onboard_repository,
                             onboard_docker,
                             show_docker_secrets,
                             show_repositories,)

conn_details = get_conn_details()
ai_core_client = AICoreV2Client(**conn_details)

## Step 0. Onboard a repository and Docker account

### GitHub repository
We'll need a GitHub username and token

In [ ]:
PROJECT_NAME = 'gsalomone-tf-gpu'
github_repo_url = 'https://github.com/gianfranco-s/SAP-aicore-gpu'
onboard_repository(github_repo_url,
                   GITHUB_USER,
                   GITHUB_TOKEN,
                   PROJECT_NAME,
                   ai_core_client)

In [3]:
show_repositories(ai_core_client)

Name: sap-training-ncellini
URL: https://github.com/NCellini/aicore-pipelines
Status: RepositoryStatus.COMPLETED

Name: gsalomone-tf-gpu
URL: https://github.com/gianfranco-s/SAP-aicore-gpu
Status: RepositoryStatus.COMPLETED



### Docker account

In [ ]:
docker_registry_name = 'gsalomone-docker'
onboard_docker(docker_registry_name, DOCKER_USER, DOCKER_TOKEN, ai_core_client)

In [2]:
show_docker_secrets(ai_core_client)

DockerRegistrySecret name: credstutorialrepo-ncellini
DockerRegistrySecret name: gsalomone-docker


## Step 1. Create workspace
Using default resource group because when I tried to create `tf-demo`, I got the message "Resource Group cannot be created for free tier tenant"

In [4]:
try:
    response = ai_core_client.resource_groups.create("tf-demo")

except Exception as e:
    print(e)

Failed to post /admin/resourceGroups: Resource Group cannot be created for free tier tenant 
 Status Code: 403, Request ID:None


In [5]:
def show_resource_groups() -> None:
    response = ai_core_client.resource_groups.query()

    for rg in response.resources:
        print(rg.resource_group_id)

show_resource_groups()

default


## Step 2: upload model files to AWS S3
Create bucket
```
aws s3api create-bucket --bucket gsalomone-celestial-bucket --region us-east-1
```

Upload files
```
cd tf_files
aws s3 cp model.h5 s3://gsalomone-celestial-bucket/movie-clf/model/  
aws s3 cp max_pad_len.txt s3://gsalomone-celestial-bucket/movie-clf/model/  
aws s3 cp label_encoded_classes.npy s3://gsalomone-celestial-bucket/movie-clf/model/  
aws s3 cp tokens.json s3://gsalomone-celestial-bucket/movie-clf/model/  
```

Check files
```
aws s3 ls s3://gsalomone-celestial-bucket/movie-clf/model/
```

## Step 3: connect AWS S3 to SAP AI Core


In [7]:
aws_credentials_file = '/home/gsalomone/Documents/gian-dev_accessKeys.csv'

def get_aws_credentials(filepath: str) -> tuple:
    with open(filepath, 'r') as f:
        creds = f.readlines()[1:2][0]
        aws_access_key_id, aws_secret_access_key = creds.split(',')
        return aws_access_key_id, aws_secret_access_key

aws_access_key_id, aws_secret_access_key = get_aws_credentials(aws_credentials_file)
aws_az = 'us-east-1'
object_store_name = 'gs-tf-gpu-tutorial-secret'

In [8]:
def create_s3_object_store() -> str:
    response = ai_core_client.object_store_secrets.create(
        resource_group = 'default',
        type = "S3",
        name = object_store_name,
        path_prefix = "movie-clf",
        endpoint = f"s3-{aws_az}.amazonaws.com",
        bucket = "gsalomone-celestial-bucket",
        region = aws_az,
        data = {
            "AWS_ACCESS_KEY_ID": aws_access_key_id,
            "AWS_SECRET_ACCESS_KEY": aws_secret_access_key
        }
    )

    return response.message

res = create_s3_object_store()
res

'secret has been created'

## Step 4: register model as Artifact
Let's follow the tutorial to the letter.

Spoiler: it will not work. **We'll follow step 6 first**

In [13]:
# Register model as artifact
from ai_core_sdk.models import Artifact

def create_model(model_name: str, object_store_name: str, scenario_id: str, description: str) -> str:
    response = ai_core_client.artifact.create(
        resource_group = 'default',
        name = model_name,
        kind = Artifact.Kind.MODEL,
        url = f"ai://{object_store_name}/model",
        scenario_id = scenario_id,  # same as under `scenarios.ai.sap.com/id`
        description = description
    )
    return response.message

try:
    msg = create_model(model_name='gsalomone-model',
                       scenario_id='tf-text-clf',
                       description='Review Classification Model',
                       object_store_name=object_store_name,)

    print(msg)
except Exception as e:
    print(e)

Failed to post /artifacts: Invalid Request, Could not create Artifact due to invalid Scenario ID tf-text-clf. Please check the Scenario ID. 
 Status Code: 400, Request ID:6591cbff-3098-4cf6-a343-4ce3ad112424


### Step 4 -> Step 6: we need to set up the scenario_id FIRST
It turns out that `scenario_id` is created from the yaml file. So we'll move to step 6 of the tutorial

- Copy [`serving_executable.yaml`](https://raw.githubusercontent.com/sap-tutorials/Tutorials/master/tutorials/ai-core-tensorflow-byod/files/workflow/serving_executable.yaml)
- set `resourcePlan` to `infer.s` which will enable GPU.
- set `imagePullSecrets > name` to the appropriate docker registry secret. In this tutorial you'll find it in variable `docker_registry_name`
- set `containers > image` is set to `"docker.io/DOCKER_USER/movie-review-clf-serve:0.0.1"`

### Step 4 -> Step 7: sync with SAP AI Core
- Push `serving_executable.yaml` to GitHub
- Create the application in SAP AI Core

In [ ]:
# Create application
application_name = 'gsalomone-tf-app'
response = ai_core_client.applications.create(
    application_name = application_name,
    revision = "HEAD",
    repository_url = github_repo_url,
    path = "tf-text-classifier"  # path to the yaml file in the repository
)

In [ ]:
response = ai_core_client.applications.get_status(application_name=application_name)

print(response.message)
for workflow_sync_status in response.sync_ressources_status:
    print(workflow_sync_status.__dict__)

AIAPIInvalidRequestException: Failed to post /artifacts: Invalid Request, Could not create Artifact due to invalid Scenario ID tf-text-clf. Please check the Scenario ID. 
 Status Code: 400, Request ID:15e41c3a-2368-4a31-a1ec-e17f9c76812a

In [ ]:
artifact_id = response.__dict__.get('id')
artifact_id

In [ ]:
# Start deployment

# Create configuration
from ai_core_sdk.models import InputArtifactBinding

response = ai_core_client.configuration.create(
    name = "gsalomone-tf-gpu-conf",
    resource_group = "default",
    scenario_id = "tf-text-clf",
    executable_id = "tf-text-clf-serve",
    input_artifact_bindings = [
        # list
        InputArtifactBinding(key="modelArtifact", artifact_id = artifact_id), # Change artifact id
    ]
)

print(response.__dict__)
config_id = response.__dict__.get('id')

In [ ]:
# Create deployment
response = ai_core_client.deployment.create(
    resource_group = "default",
    configuration_id = config_id # change this
)
print(response.__dict__)
deployment_id = response.__dict__.get('id')
deployment_id

In [ ]:
# Check deployment status
response = ai_core_client.deployment.get(
    resource_group = "default",
    deployment_id = deployment_id # Change this
)

print("Status: ", response.status)
print('*'*80)
pprint(response.__dict__)

In [ ]:
# SECOND TIME: check deployment status
response = ai_core_client.deployment.get(
    resource_group = "default",
    deployment_id = deployment_id # Change this
)

print("Status: ", response.status)
print('*'*80)
from pprint import pprint
pprint(response.__dict__)